In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

#  Submit adsorption energy

In [ ]:
# General imports.
import ipywidgets as ipw
from IPython.display import clear_output

# AiiDA imports.
%aiida

# AiiDAlab imports.
import aiidalab_widgets_base as awb

# Custom imports.
from apps.surfaces.widgets.empa_viewer import EmpaStructureViewer
from apps.surfaces.widgets.fragments import Fragment, FragmentList

from aiida_nanotech_empa.workflows.cp2k.cp2k_utils import compute_cost
from copy import copy

In [ ]:
Cp2kFragmentSeparationWorkChain = WorkflowFactory('nanotech_empa.cp2k.fragment_separation')

In [ ]:
# Structure selector.
empa_viewer = EmpaStructureViewer()
structure_selector = awb.StructureManagerWidget(
    viewer=empa_viewer,
    importers=[
        awb.StructureBrowserWidget(title="AiiDA database"),
    ],
    storable=False, node_class='StructureData')
display(structure_selector)


In [ ]:
style = {'description_width': '100px'}

In [ ]:
#UKS
uks = ipw.Checkbox(
    value=False,
    description='UKS',
    disabled=False,
    indent=False
)

#spin guess
spins_up = ipw.Text(
    value='',
    placeholder='1 2 10..13',
    description='Spins U:',
    disabled=False
)

spins_down = ipw.Text(
    value='',
    placeholder='3 4 14..17',
    description='Spins D:',
    disabled=False
)

fixed_atoms = ipw.Text(
    value='',
    placeholder='3 4 14..17',
    description='Fixed atoms:',
    disabled=False
)


#Protocol
protocol = ipw.Dropdown(
    value='standard',
    description='Protocol:',
    disabled=False,
    options = [("Standard", "standard"), ("Low accuracy", "low_accuracy"), ("Debug","debug")]
    )

In [ ]:
def get_builder_ads():
    """Get the builder for the adsorption energy calculation."""
    builder = Cp2kFragmentSeparationWorkChain.get_builder()
    builder.code = computer_code_dropdown.value
    builder.structure = structure_selector.structure_node

    # Fragments.
    builder.fragments = {
        'molecule': List(list=awb.utils.string_range_to_list(fragment_molecule.value)[0]),
        'slab': List(list=awb.utils.string_range_to_list(fragment_slab.value)[0]),
    }

    # Multiplicities.
    builder.multiplicities = {
        "total": Int(total_multiplicity.value),
        "molecule": Int(multiplicity_molecule.value),
        "slab": Int(multiplicity_slab.value),
        }
    
    # Charges.
    builder.charges = {
        "molecule": orm.Int(charge_molecule.value),
        "slab": orm.Int(charge_slab.value),
    }

    builder.options = {
        'total': {
            "max_wallclock_seconds": 600,
            "resources": {
                "num_machines": 1,
                "num_mpiprocs_per_machine": 4,
            },
        },
        'molecule': {
            "max_wallclock_seconds": 600,
            "resources": {
                "num_machines": 1,
                "num_mpiprocs_per_machine": 1,
            },
        },
        'slab': {
            "max_wallclock_seconds": 600,
            "resources": {
                "num_machines": 1,
                "num_mpiprocs_per_machine": 4,
            },
        },
    }


    # spin guess
    # ase_geom =  structure_selector.structure
    # mag_list = [ 0 for t in ase_geom ]
    # if uks.value:
    #     for i in awb.utils.string_range_to_list(spins_up.value)[0]:
    #         mag_list[i] = 1
    #     for i in awb.utils.string_range_to_list(spins_down.value)[0]:
    #         mag_list[i] = -1 
            
        
    #builder.magnetization_per_site = List(list=mag_list)


    builder.fixed_atoms = Str('3..18')
    #Charge
    builder.protocol = Str(protocol.value)


    mag = [0 for i in structure_selector.structure]
    builder.magnetization_per_site = List(list=mag)
 
    
    return builder

In [ ]:
btn_submit_ads = awb.SubmitButtonWidget(Cp2kFragmentSeparationWorkChain, 
                                input_dictionary_function=get_builder_ads
                               )

In [ ]:
fragment_list = FragmentList()
ipw.dlink((uks, 'value'), (fragment_list, 'uks'))
_ = ipw.dlink((structure_selector.viewer, 'selection'), (fragment_list, 'selection_string'), transform=lambda x: awb.utils.list_to_string_range(x))


In [ ]:
def update_resources_for_fragments(_):
    for fragment in fragment_list.fragments:
        print(fragment.indices.value)
        print(awb.utils.string_range_to_list(fragment.indices.value))
        fragment.nodes_widget.value = compute_cost(structure_selector.structure[awb.utils.string_range_to_list(fragment.indices.value)])
# Code.
computer_code_dropdown = awb.ComputationalResourcesWidget(description="CP2K code:", input_plugin="cp2k")
computer_code_dropdown.observe(update_resources_for_fragments, names='value')


In [ ]:
top = [fragment_list, uks, protocol]
bottom = [computer_code_dropdown, btn_submit_ads]

output = ipw.VBox(children=top+bottom)

def update_selection(change):

    if change['new']:
        spins_up.value = awb.utils.list_to_string_range(empa_viewer.details['spins_up'])
        spins_down.value = awb.utils.list_to_string_range(empa_viewer.details['spins_down'])

        # Add default "total" fragment.
        fragment_list.fragments = [Fragment(indices=f"1..{len(change['new'])}", name="total")]

        #print(compute_cost(change['new']))
    else:
        fragment_list.fragments = []

structure_selector.observe(update_selection, names='structure')


def update_view(_=None):
    to_display = copy(top)
    if uks.value:
        to_display += [spins_up, spins_down]
    to_display += bottom
    output.children = to_display
uks.observe(update_view, names='value')

In [ ]:
display(output)